In [50]:
import pandas as pd
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
# !pip install sklearn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# ML Libraries
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
# Global Parameters
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\c2099176\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Loading the dataset
def load_dataset(type_):
    with open(f"../data/{type_}_text.txt", "r", encoding="utf-8") as fp:
        x = fp.readlines()
    with open(f"../data/{type_}_labels.txt", "r", encoding="utf-8") as fp:
        y = [int(i) for i in fp.readlines()]
        
    data = {'text': x, 'label': y}
    df = pd.DataFrame(data)
    
    return df

In [5]:
# Pre-processing the tweets
def preprocess_tweets(tweet):
    tweet.lower()
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
    # Stemming and Lemmatizing both are normalization techniques.
    # We will use either of these not both.
    # Stemming is faster than lemmatization    
    
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    #lemmatizer = WordNetLemmatizer()
    #lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    
    return " ".join(stemmed_words)

In [6]:
# Convert Value to String
def value_to_string(sentiment):
    if sentiment == 0:
        return "Negative"
    elif sentiment == 1:
        return "Neutral"
    else:
        return "Positive"

In [7]:
def naive_bayes_model(X_train, y_train, X_any, y_any):
    # Training Naive Bayes model
    NB_model = MultinomialNB()
    NB_model.fit(X_train, y_train)
    y_predict_nb = NB_model.predict(X_any)
    print(classification_report(y_any, y_predict_nb))
#     print(accuracy_score(y_any, y_predict_nb))
    return NB_model

In [8]:
def logistic_reg_model(X_train, y_train, X_any, y_any):
    # Training Logistics Regression model
    LR_model = LogisticRegression(solver='newton-cg', max_iter=10000, multi_class="multinomial")
    LR_model.fit(X_train, y_train)
    y_predict_lr = LR_model.predict(X_any)
    print(classification_report(y_any, y_predict_lr))
#     print(accuracy_score(y_any, y_predict_lr))
    return LR_model

In [28]:
# Load train dataset
train_dataset = load_dataset("train")
old_dataset = train_dataset
# Preprocess train data
train_dataset.text = train_dataset['text'].apply(preprocess_tweets)

# Load val dataset
val_dataset = load_dataset("val")
# Preprocess val data
val_dataset.text = val_dataset['text'].apply(preprocess_tweets)

# Load test dataset
test_dataset = load_dataset("test")
# Preprocess data
test_dataset.text = test_dataset['text'].apply(preprocess_tweets)

In [10]:
vector = TfidfVectorizer(sublinear_tf=True)

In [11]:
# train
tf_vector = vector.fit(np.array(train_dataset.iloc[:, 0]).ravel())
X_train = tf_vector.transform(np.array(train_dataset.iloc[:, 0]).ravel())
y_train = np.array(train_dataset.iloc[:, 1]).ravel()

In [12]:
# validate
X_val = tf_vector.transform(np.array(val_dataset.iloc[:, 0]).ravel())
y_val = np.array(val_dataset.iloc[:, 1]).ravel()

In [13]:
# test
X_test = tf_vector.transform(np.array(test_dataset.iloc[:, 0]).ravel())
y_test = np.array(test_dataset.iloc[:, 1]).ravel()
# test_feature = tf_vector.transform(np.array(test_dataset.iloc[:, 0]).ravel())

In [14]:
naive_bayes_model(X_train, y_train, X_val, y_val)

              precision    recall  f1-score   support

           0       0.80      0.01      0.03       312
           1       0.56      0.77      0.65       869
           2       0.67      0.65      0.66       819

    accuracy                           0.60      2000
   macro avg       0.67      0.48      0.44      2000
weighted avg       0.64      0.60      0.55      2000



MultinomialNB()

In [15]:
logistic_reg_model(X_train, y_train, X_val, y_val)

              precision    recall  f1-score   support

           0       0.63      0.38      0.48       312
           1       0.63      0.74      0.68       869
           2       0.73      0.70      0.71       819

    accuracy                           0.67      2000
   macro avg       0.66      0.61      0.62      2000
weighted avg       0.67      0.67      0.66      2000



LogisticRegression(max_iter=10000, multi_class='multinomial',
                   solver='newton-cg')

In [16]:
logistic_reg_model(X_train, y_train, X_test, y_test)

              precision    recall  f1-score   support

           0       0.69      0.38      0.49      3972
           1       0.59      0.75      0.66      5937
           2       0.53      0.56      0.54      2375

    accuracy                           0.59     12284
   macro avg       0.60      0.56      0.57     12284
weighted avg       0.61      0.59      0.58     12284



LogisticRegression(max_iter=10000, multi_class='multinomial',
                   solver='newton-cg')

In [51]:
# RNN Text Classification
# !pip install tensorflow
# !pip install keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [201]:
tokenizer = Tokenizer(num_words=1500, split=' ')
# data = old_dataset
# data['text'] = data['text'].apply(lambda x: x.lower())
# data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
# print(data[ data['label'] == 2].size)
# print(data[ data['label'] == 1].size)
# print(data[ data['label'] == 0].size)
# for idx,row in data.iterrows():
#     row[0] = row[0].replace('rt',' ')

In [222]:
max_fatures = 2000
max_len = 200

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_dataset['text'].values)
# ////////////////////////
X_train = tokenizer.texts_to_sequences(train_dataset['text'].values)
X_train = pad_sequences(X_train, maxlen=max_len)
# ////////////////////////
X_val = tokenizer.texts_to_sequences(val_dataset['text'].values)
X_val = pad_sequences(X_val, maxlen=max_len)
# ////////////////////////
X_test = tokenizer.texts_to_sequences(test_dataset['text'].values)
X_test = pad_sequences(X_test, maxlen=max_len)

In [223]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 200, 128)          256000    
_________________________________________________________________
spatial_dropout1d_15 (Spatia (None, 200, 128)          0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 196)               254800    
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [224]:
Y_train = pd.get_dummies(train_dataset['label']).values
Y_val = pd.get_dummies(val_dataset['label']).values
Y_test = pd.get_dummies(test_dataset['label']).values
# X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_val.shape,Y_val.shape)
print(X_test.shape,Y_test.shape)

(45615, 200) (45615, 3)
(2000, 200) (2000, 3)
(12284, 200) (12284, 3)


In [225]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, validation_data=(X_val, Y_val), batch_size=batch_size, verbose = 2)

C:\Users\c2099176\Anaconda3\envs\py_36\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 45615 samples, validate on 2000 samples
Epoch 1/7
 - 344s - loss: 0.8348 - accuracy: 0.6121 - val_loss: 0.8129 - val_accuracy: 0.6340
Epoch 2/7
 - 343s - loss: 0.7557 - accuracy: 0.6606 - val_loss: 0.7515 - val_accuracy: 0.6690
Epoch 3/7
 - 344s - loss: 0.7288 - accuracy: 0.6704 - val_loss: 0.7342 - val_accuracy: 0.6800
Epoch 4/7
 - 358s - loss: 0.7075 - accuracy: 0.6832 - val_loss: 0.7398 - val_accuracy: 0.6780
Epoch 5/7
 - 344s - loss: 0.6884 - accuracy: 0.6923 - val_loss: 0.7391 - val_accuracy: 0.6800
Epoch 6/7
 - 343s - loss: 0.6830 - accuracy: 0.6943 - val_loss: 0.7411 - val_accuracy: 0.6715
Epoch 7/7
 - 342s - loss: 0.6582 - accuracy: 0.7065 - val_loss: 0.7574 - val_accuracy: 0.6735


In [226]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.94
acc: 0.58


In [232]:
tweet = ['i am d that the weather is not sunny!']
# vectorizing the tweet by the pre-fitted tokenizer instance
tweet = tokenizer.texts_to_sequences(tweet)
# padding the tweet to have exactly the same shape as `embedding_2` input
tweet = pad_sequences(tweet, maxlen=max_len, dtype='int32', value=0)
# print(tweet)
sentiment = model.predict(tweet, batch_size=1, verbose = 2)[0]
output = value_to_string(np.argmax(sentiment))
print(output)

Neutral
